<a href="https://colab.research.google.com/github/piyumaha12/Clootrack_assignment/blob/main/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [54]:
# !pip install googletrans
# !pip install pycld2

In [55]:
# !pip3 uninstall googletrans
# !pip3 install googletrans==3.1.0a0

In [56]:
import pandas as pd
import nltk
import re
import string
import os
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pycld2 as cld2


from sklearn.feature_extraction.text import CountVectorizer
# from spacy_langdetect import LanguageDetector
from googletrans import Translator

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
PATH = '/content/drive/MyDrive/Clootrack/'

# Dataset import

In [59]:
data = pd.read_csv(PATH+'Data Analyst - Test Data - US.csv')

data.shape

(6448, 3)

In [60]:
def read_txt(path):
    with open(path, encoding='latin') as file:
        T = file.readlines()
    T = [x.strip() for x in T]
    T = T[30:]
    return T


In [61]:
positive_words = read_txt(PATH + 'positive-words.txt')
negative_words = read_txt(PATH + 'negative-words.txt')

In [62]:
afinn = pd.read_csv(PATH +'Afinn.csv', encoding= 'latin')

In [63]:
data.head()

,Review,date,Location
0,I was very impressed with the resort.\n Great ...,2019/08/20,Sebastian
1,The rooms were nice the outside needs work als...,2019/08/20,Los Angeles
2,Great location! I have stayed at this hotel on...,2019/08/20,Georgia
3,The hotel was adequate for my stay. The strips...,2019/08/20,NaN
4,"Great location, room was large and spacious. P...",2019/08/19,Palm Harbor


In [64]:
data.tail()

,Review,date,Location
6443,"·the staff were all so friendly. frank, the ch...",2019/07/12,United States of America
6444,·the internet barely worked or didn't work,2019/07/12,United States of America
6445,·the location was great.\n \n \n \n \n ·the be...,2019/07/10,United States of America
6446,·we were early for check in and the staff was ...,2019/07/10,United States of America
6447,"·staff, free coffee, location and activities.\...",2019/07/10,United States of America


In [65]:
data.Review[:10].values

array(['I was very impressed with the resort.\n Great staff at the main resort pool bar! We had a blast with them.\n Clean, professional staff, great location and very reasonable!\n  \n  Read more\n  Read less',
       'The rooms were nice the outside needs work also no free breakfast it would have been nice overall it was ok\n  \n  Read more\n  Read less',
       'Great location! I have stayed at this hotel on my last three trips to New York.\n  \n  Read more\n  Read less',
       'The hotel was adequate for my stay. The strips in the bathtub were useless as I fell while taking a shower. I asked the front desk staff several times to get a bath mat but they never did. Management was unresponsive to my request which was very poor; someone should have gone to a dollar store and gotten a bath mat. Breakfast was good but it would have been nice to have had yogurt and/or cottage cheese on the buffet.\n  \n  Read more\n  Read less',
       'Great location, room was large and spacious. Parkin

# Basic Exploration

In [66]:
data.describe()

,Review,date,Location
count,6393,6448,1711
unique,6384,403,1082
top,Amazing\n \n Read more\n Read less,Jun 2019,United States of America
freq,4,145,116


In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6448 entries, 0 to 6447
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Review    6393 non-null   object
 1   date      6448 non-null   object
 2   Location  1711 non-null   object
dtypes: object(3)
memory usage: 151.2+ KB


In [68]:
data.isnull().sum()

Review        55
date           0
Location    4737
dtype: int64

In [69]:
data = data.dropna(subset= ['Review'])

In [70]:
data.reset_index(inplace= True, drop = True)

In [71]:
data.isnull().sum()

Review         0
date           0
Location    4688
dtype: int64

In [72]:
data.Location.value_counts()[data.Location.value_counts() >= 5]

United States of America    116
New York                     34
San Jose                     19
California                   19
Florida                      16
Canada                       16
United Kingdom               14
New York, NY                 13
Toronto                      12
Miami                        12
Orlando                      12
NY                           12
Seattle                      12
San Diego                    11
Chicago                      11
Los Angeles                  10
Philadelphia                  9
NYC                           9
Boston                        9
Maryland                      8
New Jersey                    8
London                        8
Puerto Rico                   7
Washington, DC                7
Irvine                        6
Sacramento                    6
Singapore                     6
Michigan                      6
Tampa                         6
San Francisco                 6
Atlanta, GA                   5
Vancouve

## Looking for Non ASCII lines in reviews

In [73]:
# Finding Japanese comments or non-ascii

NON_ASCII = [a.isascii() for a in data.Review]
NON_ASCII = pd.Series(NON_ASCII)

In [74]:
data[NON_ASCII == False]

,Review,date,Location
8,My wife and I loved everything about it. It’s ...,2019/08/19,NaN
10,Very clean and friendly and I love the locatio...,2019/08/19,California
18,"It was great for what we needed, a place to sl...",2019/08/19,NaN
21,The room is tiny. The building is old and worn...,2019/08/18,NaN
23,基本的に問題なしでした。\n 清潔で防音も普通な方で、水回りも清潔でシャワーの水圧も問題なし...,2019/08/18,NaN
...,...,...,...
6388,"·the staff were all so friendly. frank, the ch...",2019/07/12,United States of America
6389,·the internet barely worked or didn't work,2019/07/12,United States of America
6390,·the location was great.\n \n \n \n \n ·the be...,2019/07/10,United States of America
6391,·we were early for check in and the staff was ...,2019/07/10,United States of America


In [75]:
NON_ASCII.value_counts()

True     4390
False    2003
dtype: int64

# Data Cleaning

First of all we have to clean the dataset and then after we go for visualization and exploration.

We need to perform following tasks
1. Remove new lines
2. Remore "Read more" and "Read less"


In [76]:
def data_cleaning(strn):
    '''
    This function removes following things:
    - Remove emojis
    - Remove excessive blank spaces
    
    '''
    punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    strn = strn.encode('ascii', 'ignore').decode('ascii')
    strn = re.sub(r'http\S+\s*', '', strn)
    strn = re.sub(r'[#@]\w', '', strn)
    strn = strn.replace('\n', '')
    strn = strn.replace('Read more', '')
    strn = strn.replace('Read less', '')
    strn = re.sub(r' {2,}', ' ', strn)
    
    # strn = strn.replace('.', '') if strn[0][0] == '.' else strn
    strn = strn.strip()
    return strn


In [77]:
data['Review']=data['Review'].apply(data_cleaning).values

In [78]:
data.Review.values[:10]

array(['I was very impressed with the resort. Great staff at the main resort pool bar! We had a blast with them. Clean, professional staff, great location and very reasonable!',
       'The rooms were nice the outside needs work also no free breakfast it would have been nice overall it was ok',
       'Great location! I have stayed at this hotel on my last three trips to New York.',
       'The hotel was adequate for my stay. The strips in the bathtub were useless as I fell while taking a shower. I asked the front desk staff several times to get a bath mat but they never did. Management was unresponsive to my request which was very poor; someone should have gone to a dollar store and gotten a bath mat. Breakfast was good but it would have been nice to have had yogurt and/or cottage cheese on the buffet.',
       'Great location, room was large and spacious. Parking was easy and we appreciated the friendly staff',
       'All the options for fun were great! Do wish there were a better p

> Now the dataset is almost ready and clean
>
> Only one thing is to remove is Punctuation marks

In [79]:
data.isnull().sum()

Review         0
date           0
Location    4688
dtype: int64

In [80]:
data[data.Review == '']

,Review,date,Location
23,,2019/08/18,NaN
311,,2019/07/20,NaN
505,,2019/07/05,NaN
693,,2019/06/15,大阪
723,,2019/06/10,東京
1026,,2019/05/12,NaN
1073,,2019/05/08,NaN
1111,,2019/05/05,NaN
1129,,2019/05/04,NaN
1258,,2019/04/26,JAPAN


In [81]:
data[data.Review == ''].value_counts()

Review  date        Location       
        2018/08/23  Shizuoka, Japan    1
        2019/04/26  JAPAN              1
        2019/06/10  東京                 1
        2019/06/15  大阪                 1
        2019/07/20  Kuwait             1
        2019/1/8    nagoya, jp         1
dtype: int64

As You can see all Non-English comments are blank to lets remove them

In [82]:
data = data[data.Review != '']

In [83]:
data.reset_index(inplace= True, drop = True)

In [84]:
data

,Review,date,Location
0,I was very impressed with the resort. Great st...,2019/08/20,Sebastian
1,The rooms were nice the outside needs work als...,2019/08/20,Los Angeles
2,Great location! I have stayed at this hotel on...,2019/08/20,Georgia
3,The hotel was adequate for my stay. The strips...,2019/08/20,NaN
4,"Great location, room was large and spacious. P...",2019/08/19,Palm Harbor
...,...,...,...
6348,"the staff were all so friendly. frank, the che...",2019/07/12,United States of America
6349,the internet barely worked or didn't work,2019/07/12,United States of America
6350,the location was great. the bed was the most u...,2019/07/10,United States of America
6351,we were early for check in and the staff was v...,2019/07/10,United States of America


In [85]:
data['Review_cleaned'] = data['Review'].apply(lambda x: re.sub('[%s]'%re.escape(string.punctuation), ' ', x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [86]:
data.Review_cleaned[:5].values

array(['I was very impressed with the resort  Great staff at the main resort pool bar  We had a blast with them  Clean  professional staff  great location and very reasonable ',
       'The rooms were nice the outside needs work also no free breakfast it would have been nice overall it was ok',
       'Great location  I have stayed at this hotel on my last three trips to New York ',
       'The hotel was adequate for my stay  The strips in the bathtub were useless as I fell while taking a shower  I asked the front desk staff several times to get a bath mat but they never did  Management was unresponsive to my request which was very poor  someone should have gone to a dollar store and gotten a bath mat  Breakfast was good but it would have been nice to have had yogurt and or cottage cheese on the buffet ',
       'Great location  room was large and spacious  Parking was easy and we appreciated the friendly staff'],
      dtype=object)

# Lemmitization

In [87]:
nlp = spacy.load('en_core_web_sm')

data['lemmatized'] = data['Review'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop == False)]))
data['lemmatized'] = [x.strip() for x in data['lemmatized']]

# Language Detection

In [116]:
T = data.Review[0]

In [117]:
T

'I was very impressed with the resort. Great staff at the main resort pool bar! We had a blast with them. Clean, professional staff, great location and very reasonable!'

In [118]:
_,_,_, detected_language = cld2.detect(T, returnVectors = True )

In [119]:
detected_language[0][2]

'ENGLISH'

In [128]:
def language_detection(strn):
  _,_,_, detected_lang = cld2.detect(strn, returnVectors = True)
  if len(detected_lang) > 1:
    langs = []
    for i in detected_lang:
      lang = i[2]
      langs.append(lang)
  else:
    try:
      langs = detected_lang[0][2]
    except:
      langs = None
  return langs

In [130]:
data['Languages']=data.Review.apply(language_detection)

In [133]:
data['Languages'].isnull().sum()

23

# Sentiment score using counts of positive and negative words

In [38]:
affinity_score = afinn.set_index('word')['value'].to_dict()

nlp = spacy.load('en_core_web_sm')
sentiment_lexicon = affinity_score

def calculate_sentiment(text: str = None):
    sent_score = 0
    if text:
        sentence = nlp(text)
        for word in sentence:
            sent_score += sentiment_lexicon.get(word.lemma_, 0)
    return sent_score
    
data['Sentiment'] = data['Review_cleaned'].apply(calculate_sentiment)

In [39]:
px.histogram(data_frame= data, x = 'Sentiment')

In [40]:
data[data.Sentiment < -10 ]

,Review,date,Location,Review_cleaned,lemmatized,Sentiment
653,Do not valet your vehicle at this property! I ...,2019/06/21,NaN,Do not valet your vehicle at this property I ...,valet vehicle property ! mistake allow valet m...,-13
665,Ich war groer Fax des Hotels und war bereits G...,2019/06/19,NaN,Ich war groer Fax des Hotels und war bereits G...,Ich war groer Fax des Hotels und war bereit Ga...,-33
751,This was a terrible experience to me. First of...,2019/06/04,"Montreal, Canada",This was a terrible experience to me First of...,terrible experience . book night stay hotel sh...,-19
940,"Ursprnglich gutes Angebot, spter Ernchterung H...",2019/05/17,NaN,Ursprnglich gutes Angebot spter Ernchterung H...,"ursprnglich gut Angebot , spter Ernchterung Ho...",-31
950,Carpets in hallways were gross & dirty. Same p...,2019/05/16,Visalia Ca,Carpets in hallways were gross dirty Same p...,carpet hallway gross & dirty . picture post ha...,-12
1289,"Das Hotel ist lter, aber in einem sauberen Zus...",2019/04/23,"Linz, at",Das Hotel ist lter aber in einem sauberen Zus...,"Das Hotel ist lter , aber einem sauberen Zusta...",-13
1884,Handicap room this sucks I hate the shower I h...,2019/03/24,Modesto ca,Handicap room this sucks I hate the shower I h...,handicap room suck hate shower hate hate room,-12
1906,We were evacuated twice from the hotel. The fi...,2019/03/22,NaN,We were evacuated twice from the hotel The fi...,evacuate twice hotel . time Wednesday night 10...,-12
1908,Die Mitarbeiter waren sehr nett und hilfsberei...,2019/03/22,NaN,Die Mitarbeiter waren sehr nett und hilfsberei...,die Mitarbeiter waren sehr nett und hilfsberei...,-19
1916,"The condition of the rooms were very bad, dirt...",2019/03/22,NaN,The condition of the rooms were very bad dirt...,"condition room bad , dirty smell . bathroom fl...",-15
